
Revathi's tasks:
redo yohannes EDA with our csv
do 2 different logistic regressions on ERIOD and SENTIMP as targets
research on time periods
1 = Koon Period (June 13, 1996 through April 30, 2003)
2 = PROTECT Act Period (May 1, 2003 through June 24, 2004)
3 = Booker Period (January 12, 2005 through December 10, 2007)
4 = Gall Period (December 11, 2007 through September 30, 2011)


In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings
import os
from category_encoders import TargetEncoder 
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline

from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from skopt import BayesSearchCV
import pickle

In [9]:
#reading in file 
booker=pd.read_csv('Datasets/eda_booker_report.csv')
booker=booker.iloc[:,1:]
booker

,ACCAP,AGE,GGDUM,BOOKER2,AROFFAP,CIRCDIST,MITDUM,MONCIRC,ONSEX,NEWCIT,...,SAFEVALVE,SENTIMP,OTCHPTS,WEAPON,XCRHISSR,LMIN,FY,USSCIDN,ENSPLT0,GDL
0,0,28,0,2,0,80.0,0,10.0,0,1,...,2,1,5,0,3,10,2005,890751.0,8.0,0
1,0,24,0,2,0,80.0,0,10.0,0,0,...,2,1,3,0,2,168,2005,890752.0,72.0,1
2,0,32,0,0,0,33.0,0,5.0,1,0,...,0,1,3,0,2,12,2005,890755.0,15.0,2
3,0,24,1,0,0,33.0,0,5.0,0,0,...,0,1,10,1,5,228,2005,890756.0,270.0,2
4,0,43,0,0,0,93.0,0,11.0,0,1,...,2,1,9,0,4,57,2005,890757.0,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,0,40,0,2,0,47.0,0,7.0,1,1,...,2,4,0,0,1,6,2004,759721.0,0.0,4
799996,0,41,0,0,0,47.0,0,7.0,0,1,...,2,1,0,0,1,24,2004,759722.0,24.0,51
799997,0,23,0,0,0,47.0,0,7.0,0,0,...,2,3,1,0,1,6,2004,759723.0,6.0,8
799998,0,30,0,0,0,47.0,0,7.0,0,1,...,2,1,8,0,4,57,2004,759724.0,61.0,0


In [14]:
booker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 28 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ACCAP      800000 non-null  int64  
 1   AGE        800000 non-null  int64  
 2   GGDUM      800000 non-null  int64  
 3   BOOKER2    800000 non-null  int64  
 4   AROFFAP    800000 non-null  int64  
 5   CIRCDIST   800000 non-null  float64
 6   MITDUM     800000 non-null  int64  
 7   MONCIRC    800000 non-null  float64
 8   ONSEX      800000 non-null  int64  
 9   NEWCIT     800000 non-null  int64  
 10  NEWCNVTN   800000 non-null  int64  
 11  EWEDUC     800000 non-null  int64  
 12  NEWRACE    800000 non-null  int64  
 13  OFFTYPE2   800000 non-null  int64  
 14  ERIOD      800000 non-null  int64  
 15  PRIMARY    800000 non-null  int64  
 16  QUARTER    800000 non-null  float64
 17  A          800000 non-null  int64  
 18  SAFEVALVE  800000 non-null  int64  
 19  SENTIMP    800000 non-n

In [31]:
# defining X and y and splitting into traintest sets
y= booker['SENTIMP']
X = booker.drop(columns = 'SENTIMP', 'ENSPLT0')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

In [32]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Logistic Regression

In [33]:
lg = LogisticRegression(random_state = 42)
lg.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [34]:
lg.score(X_train,y_train)

0.9151964285714286

In [35]:
cross_val_score(lg, X_train, y_train, cv=5).mean()

0.9141142857142857

In [36]:
y_pred = lg.predict(X_test)


In [37]:
lg.score(X_test,y_test)

0.9161208333333334

## BayesSearchCV using Logistic Regression

In [46]:
lg = LogisticRegression(random_state =42)


pipe_lg_params = {
    'C': np.logspace(-4, 4, 50), 
    'max_iter': [10_000, 1_000_000]
}   

bs_lg=BayesSearchCV(
    estimator=lg,
    search_spaces=pipe_lg_params,
    n_iter=50, 
    verbose=1,
    cv=5,
    n_jobs=-1
)

In [47]:
bs_lg.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

BayesSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
              search_spaces={'C': array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095476e-04, 1.38949549e-03,
       2.02358965e-03, 2.94705170e-03, 4.29193426e-03, 6.25055193e-03,
       9.10298178e-03, 1.32571137e-02, 1.93069773e-02, 2.81176870e-02,
       4.0949150...
       3.72759372e+00, 5.42867544e+00, 7.90604321e+00, 1.15139540e+01,
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04]),
                             'max_iter': [10000, 1000000]},
              verbose=1)

In [48]:
bs_lg.score(X_train,y_train)

0.917375

In [57]:
cross_val_score(bs_lg, X_train, y_train, cv=5).mean()

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

0.9174142857142857

In [51]:
y_pred = bs_lg.predict(X_test)

In [52]:
bs_lg.score(X_test,y_test)

0.9181375

## GridSearchCV using Logistic Regression

In [53]:
parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100, 1000]}]
gs = GridSearchCV(estimator = lg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


gs.fit(X_train, y_train) 

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
             param_grid=[{'penalty': ['l1', 'l2']}, {'C': [1, 10, 100, 1000]}],
             scoring='accuracy')

In [54]:
gs.score(X_train,y_train)

0.9147196428571429

In [58]:
cross_val_score(gs, X_train, y_train, cv=5).mean()

0.9146678571428571

In [59]:
y_pred = gs.predict(X_test)

In [60]:
gs.score(X_test,y_test)

0.9156583333333334

## Random Forest Classifier  

In [64]:
# instantiate and fit model
rf = RandomForestClassifier(n_estimators=200, 
                            max_depth=None, 
                            max_features='log2', 
                            min_samples_split=50)
rf.fit(X_train, y_train)
# scoring train model data
rf.score(X_train, y_train)

0.9528035714285714

In [65]:
cross_val_score(rf,X_train,y_train,cv=5).mean()

0.9439

In [66]:
rf.score(X_test,y_test)

0.9446708333333333

## KNN Classifier N_Neighbors = 3

In [71]:
# knn3 = KNeighborsClassifier(n_neighbors=3)

In [72]:
# knn3.fit(X_train,y_train) 

In [73]:
# knn3.score(X_train,y_train)

In [74]:
# knn3.score(X_test,y_test)

In [75]:
# cross_val_score(knn3,X_train,y_train,cv=3).mean()

k=5

## KNN Classifier N_Neighbors = 5

In [76]:
# knn5=KNeighborsClassifier(n_neighbors=5)

In [77]:
# knn5.fit(X_train,y_train)

In [78]:
# knn5.score(X_train,y_train)

In [79]:
# knn5.score(X_test,y_test)

In [80]:
# cross_val_score(knn5,X_train,y_train,cv=3).mean()

k=7

## KNN Classifier N_Neighbors = 7

In [81]:
# knn7=KNeighborsClassifier(n_neighbors=7)

In [82]:
# knn7.fit(X_train,y_train)

In [83]:
# knn7.score(X_train,y_train)

In [84]:
# knn7.score(X_test,y_test)

In [85]:
# cross_val_score(knn7,X_train,y_train,cv=3).mean()

k=9

## KNN Classifier N_Neighbors = 9

In [86]:
# knn9 = KNeighborsClassifier(n_neighbors=9)

In [87]:
# knn9.fit(X_train, y_train)

In [88]:
# knn9.score(X_train, y_train)

In [89]:
# knn9.score(X_test, y_test)

In [90]:
# cross_val_score(knn9, X_train, y_train, cv = 3).mean()

In [97]:
with open('sentimp_bs_lg', 'wb') as pickle_out:
    pickle.dump(bs_lg, pickle_out)

In [101]:
with open('sentimp_gs_lg', 'wb') as pickle_out:
    pickle.dump(gs, pickle_out)

In [100]:
with open('sentimp_rf', 'wb') as pickle_out:
    pickle.dump(rf, pickle_out)